In [ ]:
import pandas as pd 
import numpy as np

### 절대모멘텀
1. 주식 데이터 로드 
2. Date를 시계열로 변경
3. 파생변수 생성 (년월) -> Date에서 년-월을 추출
4. 년, 월 별 마지막 날의 month_last_df 데이터프레임을 하나 생성
5. 전월의 종가를 가지는 파생변수를 생성 (month_last_df)
6. 전년도의 종가를 가지는 파생변수 생성 (month_last_df)
7. 전월, 전년도의 종가를 가지고 거래 내역 생성
8. 수익율 계산 
9. 누적 수익율 계산

In [ ]:
df = pd.read_csv('../../csv/GDX.csv')
df2 = df.copy()

In [ ]:
df2.set_index('Date', inplace=True)

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df2.index = pd.to_datetime(df2.index, format='%Y-%m-%d')

In [ ]:
# 파생변수 이름은 STD-YM
# 년-월 Date 컬럼에서 추출

# case1
df['Date'].dt.strftime('%Y-%m')

In [ ]:
# case2 (apply() + 일반함수)
def change(x):
    # x는? -> Date의 value -> type : datetime
    result = x.strftime('%Y-%m')
    return result

df['Date'].apply(change)

In [ ]:
# case3 (apply() + lambda함수)

df['STD-YM'] = df['Date'].apply(lambda x : x.strftime('%Y-%m'))
df.set_index('Date', inplace=True)

In [ ]:
# 시계열 데이터가 인덱스에 존재할때
df2['STD-YM'] = df2.index.strftime('%Y-%m').to_list()

In [ ]:
df.head(3)

In [ ]:
df2.head(3)

In [ ]:
# 말일의 데이터를 모아서 하나의 데이터프레임으로 생성4

# case1 (shift사용)
flag = df['STD-YM'] != df.shift(-1)['STD-YM']
df.loc[flag]

In [ ]:
df.loc[df['STD-YM'] == '2009-01'].tail(1)

In [ ]:
# case2 (for문)

month_last_df = pd.DataFrame()


# 년월 데이터들들을 필터링 -> 가장 마지막의 데이터를 test_df 추가

# STD-YM 중복된 데이터를 없애고 리스트 출력
_list = df['STD-YM'].unique()
# df['STD-YM'].value_counts().index

for i in _list:
    last_data = df.loc[df['STD-YM'] == i].iloc[[-1]]
    month_last_df = pd.concat([month_last_df, last_data], axis=0)

month_last_df


In [ ]:
# 파생변수 BF_1M 생성하여 전월의 수정종가(Adj Close)의 값을 대입(결측치는 0으로 대체)
month_last_df['BF_1M'] =  month_last_df['Adj Close'].shift(1).replace(np.nan, 0)

In [ ]:
# 파생변수 BF_12M 생성하여 전년도의 수정종가(Adj Close)의 값을 대입(결측치는 0으로 대체)
month_last_df['BF_12M'] =  month_last_df['Adj Close'].shift(12).fillna(0)

In [ ]:
month_last_df.iloc[10:15]

In [ ]:
# 거래 내역 추가 (df)
df['trade'] = ""
# 구매 조건 -> ((전월 종가 / 전 년도의 종가) - 1)의 값이 0보다 크고 무한대 아닌 경우
for i in month_last_df.index:
    # 구매 상태 변수를 생성
    signal = ""

    # 절대 모멘텀을 계산
    momentum_index = month_last_df.loc[i, 'BF_1M'] \
        / month_last_df.loc[i, 'BF_12M'] - 1
    # print(momentum_index)
    # momentum_index가 0보다 크고 무한대가 아닌 경우 signal을 buy
    # 조건식 생성
    # case1
    flag = True if ((momentum_index > 0) and (momentum_index != np.inf) 
                    and (momentum_index != -np.inf)) else False
    # case2
    # if (momentum_index > 0) and (momentum_index != np.inf)\
    #       and (momentum_index != -np.inf):
    #     flag = True
    # else:
    #     flag = False

    if flag:
        signal = 'buy'
    
    print('날짜 :', i, "모멘텀 인덱스 :", momentum_index, 
          'flag :', flag, 'signal :', signal)
    df.loc[i, 'trade'] = signal
    

In [ ]:
df['trade'].value_counts()

### 수익율 계산
1. 파생변수(return) 생성 ->  데이터는 1
2. rtn 변수를 생성하여 1 초기값을 지정 
3. buy, sell = 0 으로 초기 값을 지정 
4. 반복문을 이용하여 현재 행의 trade가 buy이고 전 행의 trade가 ""인 경우 구매가에 수정종가를 대입
5. 전 행의 trade가 'buy'이고 현재의 행이 ''인 경우 판매가에 수정종가를 대입
6. 수익율 계산하여 대입
### 누적 수익율 계산
1. acc_rtn = 1 생성
2. return의 항목들을 누적 곱하여 acc_rtn 컬럼에 대입 
3. acc_rtn 출력

In [ ]:
df['return'] = 1
rtn = 1
buy = 0
sell = 0

for i in df.index:
    # 구매한 날을 체크 
    if (df.loc[i, 'trade'] == 'buy') and (df.shift(1).loc[i, 'trade'] == ""):
        buy = df.loc[i, 'Adj Close']
        print('매수일 :', i, "매수가 :", buy)
    # 판매한 날을 체크
    elif (df.loc[i, 'trade'] == "") and (df.shift(1).loc[i, 'trade'] == 'buy'):
        sell = df.loc[i, 'Adj Close']
        # 수익율 계산
        rtn = sell / buy
        # 수익율을 데이터프레임에 대입
        df.loc[i, 'return'] = rtn
        print('매도일 :', i, "매도가 :", sell, '수익율 :', rtn)

In [ ]:
acc_rtn = 1

for i in df.index:
    acc_rtn *= df.loc[i, 'return']
    df.loc[i, 'acc_rtn'] = acc_rtn

print(acc_rtn)

### 절대 모멘텀 함수화 
1. 첫 번째 함수
    - 매개변수 1개 (데이터프레임)
    - Date라는 컬럼이 존재하는가? -> 존재하면 Date를 인덱스로 변경
    - 수정종가를 제외한 나머지 컬럼을 삭제
    - 데이터프레임에서 결측치와 무한대를 제거 
    - 인덱스를 시계열 변경 
    - 'STD-YM' 파생변수 생성하여 인덱스의 '년-월' 추출하여 대입
    - 데이터프레임을 리턴
2. 두 번째 함수
    - 매개변수 1개 (데이터프레임)
    - 새로운 데이터프레임을 생성
    - 인자값으로 받아온 데이터프레임에서 년-월별 마지막 데이터를 새로운 데이터프레임에 대입
    - 새로운 데이터프레임에 파생변수 2개 생성 
        - 'BF_1M' : 전월의 종가, 결측치는 0으로 대체
        - 'BF_12M' : 전 년도의 종가, 결측치는 0으로 대체
    - 새로운 데이터 프레임을 리턴
3. 세 번째 함수
    - 매개변수 2개(1번의 결과(df1), 2번 결과(df2))
    - df1에 trade 파생변수 생성 -> 데이터는 ''
    - df1에 return 파생변수 생성 -> 데이터는 1
    - df2의 값들을 이용하여 momentum_index를 구하고 df1에 거래내역을 삽입
    - df1의 거래내역을 이용하여 수익율 return에 대입
    - return컬럼의 데이터를 가지고 누적 수익율(acc_rtn)에 대입 
    - 총 누적 수익율을 출력
    - df1을 리턴

In [4]:
df = pd.read_csv('../../csv/AAPL.csv')

In [3]:
from datetime import datetime
import pandas as pd
import numpy as np

In [5]:
# 첫 번째 함수 
def add_col(df, col = 'Close', start="20080101", end = "20231231"):
    # Date 라는 컬럼이 존재하는가?
    if 'Date' in df.columns:
        df.set_index('Date', inplace=True)

    start = int(start) - 10000
    start = datetime.strptime(str(start), '%Y%m%d').isoformat()
    end = datetime.strptime(end, '%Y%m%d').isoformat()
    
    
    # 수정종가를 제외한 나머지 컬럼은 제거 
    df = df[[col]]

    # 결측치와 무한대가 아닌 경우 조건식 생성
    flag = ~(df.isin([np.nan, np.inf, -np.inf]).any(axis=1))
    df = df.loc[flag]

    # 인덱스를 시계열 데이터로 변경 
    df.index = pd.to_datetime(df.index, format='%Y-%m-%d')

    df = df.loc[start:end]

    # 인덱스의 시계열중 '년-월' 추출하여 STD-YM에 대입
    df['STD-YM'] = (df.index.strftime('%Y-%m')).to_list()

    return df

In [6]:
momentum_data = add_col(df, 'Adj Close')

In [7]:
momentum_data

,Adj Close,STD-YM
Date,,
2007-01-03,10.487321,2007-01
2007-01-04,10.720094,2007-01
2007-01-05,10.643752,2007-01
2007-01-08,10.696313,2007-01
2007-01-09,11.584859,2007-01
...,...,...
2019-06-18,198.449997,2019-06
2019-06-19,197.869995,2019-06
2019-06-20,199.460007,2019-06


In [8]:
def add_df(df):
    # 기준이 되는 컬럼
    col = df.columns[0]
    # case1
    # 새로운 컬럼을 생성
    # df2 = pd.DataFrame()

    # # STD-YM데이터의 중복된 데이터를 제거하고 리스트 형태로 변수에 대입
    # _list = df['STD-YM'].unique()

    # # _list를 기준으로 반복문을 사용
    # for i in _list:
    #     last_data = df.loc[df['STD-YM'] == i].tail(1)
    #     df2 = pd.concat([df2, last_data], axis=0)

    # case2 인덱스의 조건식으로 말일의 데이터를 출력(현재행의 STD-YM과 다음 행의 STD-YM이 다른 경우)
    flag = df['STD-YM'] != df.shift(-1)['STD-YM']
    df2 = df.loc[flag]

    # 전월의 종가를 BF_1M에 대입
    df2['BF_1M'] = df2.shift(1)[col].fillna(0)
    # 전 년도의 종가를 BF_12M에 대입
    df2['BF_12M'] = df2.shift(12)[col].fillna(0)

    return df2

In [9]:
momentum_data2 = add_df(momentum_data)

C:\Users\moons\AppData\Local\Temp\ipykernel_1756\3795696707.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['BF_1M'] = df2.shift(1)[col].fillna(0)
C:\Users\moons\AppData\Local\Temp\ipykernel_1756\3795696707.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['BF_12M'] = df2.shift(12)[col].fillna(0)


In [10]:
momentum_data2

,Adj Close,STD-YM,BF_1M,BF_12M
Date,,,,
2007-01-31,10.728851,2007-01,0.000000,0.000000
2007-02-28,10.588690,2007-02,10.728851,0.000000
2007-03-30,11.627406,2007-03,10.588690,0.000000
2007-04-30,12.489673,2007-04,11.627406,0.000000
2007-05-31,15.166566,2007-05,12.489673,0.000000
...,...,...,...,...
2019-02-28,172.485748,2019-02,165.093445,174.775360
2019-03-29,189.221313,2019-03,172.485748,164.629501
2019-04-30,199.900192,2019-04,189.221313,162.156830


In [107]:
# 세 번째 함수
def add_rtn(df1, df2):
    # df1에 trade와 return 추가 
    df1['trade'] = ""
    df1['return'] = 1


    # 모멘텀 지수를 이용하여 거래내역을 추가 
    for i in df2.index:
        signal = ""
        # 모멘텀 인덱스를 변수에 대입
        momentum_index = df2.loc[i, 'BF_1M'] / df2.loc[i, 'BF_12M'] - 1
        # 모멘텀 인덱스가 0보다 크고 무한대가 아니라면 구매
        flag = True if((momentum_index > 0) and 
                        (momentum_index != np.inf) and 
                        (momentum_index != -np.inf)) else False
        if flag:
            signal = 'buy'
        # 모멘텀 인덱스를 기준으로 구매 내역을 추가
        df1.loc[i, 'trade'] = signal

    # 수익율을 계산 
    rtn = 1 
    buy = 0 
    sell = 0
    # 기준이 되는 컬럼의 이름
    col = df1.columns[0]
    for i in df1.index:
        # 구매 한 날이라면? -> 전날의 trade가 ""이고 오늘의 trade가 'buy'인 경우
        if (df1.loc[i, 'trade'] == 'buy') and (df1.shift(1).loc[i, 'trade'] == ''):
            buy = df1.loc[i, col]
            print('매수일 : ', i, "매수가 : ", buy)
        # 판매한 날이라면? -> 전날의 trade가 "buy"이고 오늘의 trade가 ''인 경우
        elif (df1.loc[i, 'trade'] == '') and (df1.shift(1).loc[i, 'trade'] == 'buy'):
            sell = df1.loc[i, col]
            # 수익율 계산
            rtn = sell / buy
            print('매도일 : ', i, "매도가 :", sell, '수익율 :', rtn)
            df1.loc[i, 'return'] = rtn
    
    # 누적 수익율을 계산
    acc_rtn = 1

    for i in df1.index:
        acc_rtn *= df1.loc[i, 'return']
        df1.loc[i, 'acc_rtn'] = acc_rtn
    print(acc_rtn)
    
    return df1

In [108]:
add_rtn(momentum_data, momentum_data2)

C:\Users\moons\AppData\Local\Temp\ipykernel_4284\166527764.py:12: RuntimeWarning: invalid value encountered in scalar divide
  momentum_index = df2.loc[i, 'BF_1M'] / df2.loc[i, 'BF_12M'] - 1
C:\Users\moons\AppData\Local\Temp\ipykernel_4284\166527764.py:12: RuntimeWarning: divide by zero encountered in scalar divide
  momentum_index = df2.loc[i, 'BF_1M'] / df2.loc[i, 'BF_12M'] - 1


매수일 :  1982-09-30 00:00:00 매수가 :  0.260594
매도일 :  1982-10-01 00:00:00 매도가 : 0.264164 수익율 : 1.0136994712080862
매수일 :  1982-11-30 00:00:00 매수가 :  0.455147
매도일 :  1982-12-01 00:00:00 매도가 : 0.464072 수익율 : 1.019609049384045
매수일 :  1982-12-31 00:00:00 매수가 :  0.426589
매도일 :  1983-01-03 00:00:00 매도가 : 0.406955 수익율 : 0.9539744344087635
매수일 :  1983-01-31 00:00:00 매수가 :  0.58366
매도일 :  1983-02-01 00:00:00 매도가 : 0.596154 수익율 : 1.0214062981873009
매수일 :  1983-02-28 00:00:00 매수가 :  0.651485
매도일 :  1983-03-01 00:00:00 매도가 : 0.662195 수익율 : 1.0164393654497033
매수일 :  1983-03-31 00:00:00 매수가 :  0.603293
매도일 :  1983-04-04 00:00:00 매도가 : 0.587229 수익율 : 0.9733728055853458
매수일 :  1983-04-29 00:00:00 매수가 :  0.721096
매도일 :  1983-05-02 00:00:00 매도가 : 0.699677 수익율 : 0.9702966040582669
매수일 :  1983-05-31 00:00:00 매수가 :  0.82462
매도일 :  1983-06-01 00:00:00 매도가 : 0.829974 수익율 : 1.006492687540928
매수일 :  1983-06-30 00:00:00 매수가 :  0.697892
매도일 :  1983-07-01 00:00:00 매도가 : 0.703247 수익율 : 1.0076731070136926
매수일 :  1983-07

,Adj Close,STD-YM,trade,return,acc_rtn
Date,,,,,
1980-12-12,0.410525,1980-12,,1.0,1.000000
1980-12-15,0.389106,1980-12,,1.0,1.000000
1980-12-16,0.360548,1980-12,,1.0,1.000000
1980-12-17,0.369472,1980-12,,1.0,1.000000
1980-12-18,0.380182,1980-12,,1.0,1.000000
...,...,...,...,...,...
2019-06-18,198.449997,2019-06,,1.0,2.229145
2019-06-19,197.869995,2019-06,,1.0,2.229145
2019-06-20,199.460007,2019-06,,1.0,2.229145


In [12]:
import momentum
import pandas as pd

In [13]:
df = pd.read_csv("../../csv/AAPL.csv")

In [15]:
data1 = momentum.add_col(df)

In [17]:
data2 = momentum.add_df(data1)

c:\Users\moons\Documents\GitHub\ubion-8\python\230821\momentum.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['BF_1M'] = df2.shift(1)[col].fillna(0)
c:\Users\moons\Documents\GitHub\ubion-8\python\230821\momentum.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['BF_12M'] = df2.shift(12)[col].fillna(0)


In [18]:
momentum.add_rtn(data1, data2)

c:\Users\moons\Documents\GitHub\ubion-8\python\230821\momentum.py:71: RuntimeWarning: invalid value encountered in scalar divide
  momentum_index = df2.loc[i, 'BF_1M'] / df2.loc[i, 'BF_12M'] - 1
c:\Users\moons\Documents\GitHub\ubion-8\python\230821\momentum.py:71: RuntimeWarning: divide by zero encountered in scalar divide
  momentum_index = df2.loc[i, 'BF_1M'] / df2.loc[i, 'BF_12M'] - 1


매수일 :  2000-01-31 00:00:00 매수가 :  3.705357
매도일 :  2000-02-01 00:00:00 매도가 : 3.580357 수익율 : 0.9662650589403396
매수일 :  2000-02-29 00:00:00 매수가 :  4.09375
매도일 :  2000-03-01 00:00:00 매도가 : 4.654018 수익율 : 1.136859358778626
매수일 :  2000-03-31 00:00:00 매수가 :  4.850446
매도일 :  2000-04-03 00:00:00 매도가 : 4.761161 수익율 : 0.9815924143882853
매수일 :  2000-04-28 00:00:00 매수가 :  4.430804
매도일 :  2000-05-01 00:00:00 매도가 : 4.439732 수익율 : 1.0020149841879713
매수일 :  2000-05-31 00:00:00 매수가 :  3.0
매도일 :  2000-06-01 00:00:00 매도가 : 3.183036 수익율 : 1.061012
매수일 :  2000-06-30 00:00:00 매수가 :  3.741071
매도일 :  2000-07-03 00:00:00 매도가 : 3.808036 수익율 : 1.0178999543178946
매수일 :  2000-07-31 00:00:00 매수가 :  3.629464
매도일 :  2000-08-01 00:00:00 매도가 : 3.522321 수익율 : 0.9704796631127901
매수일 :  2000-08-31 00:00:00 매수가 :  4.352679
매도일 :  2000-09-01 00:00:00 매도가 : 4.53125 수익율 : 1.041025538524665
매수일 :  2000-09-29 00:00:00 매수가 :  1.839286
매도일 :  2000-10-02 00:00:00 매도가 : 1.732143 수익율 : 0.9417475041945624
매수일 :  2001-11-30 00:00:00 매수

,Close,STD-YM,trade,return,acc_rtn
Date,,,,,
1999-01-04,1.473214,1999-01,,1.0,1.000000
1999-01-05,1.546875,1999-01,,1.0,1.000000
1999-01-06,1.491071,1999-01,,1.0,1.000000
1999-01-07,1.607143,1999-01,,1.0,1.000000
1999-01-08,1.607143,1999-01,,1.0,1.000000
...,...,...,...,...,...
2019-06-18,198.449997,2019-06,,1.0,2.077912
2019-06-19,197.869995,2019-06,,1.0,2.077912
2019-06-20,199.460007,2019-06,,1.0,2.077912
